In [1]:
import numpy as np
import seaborn as sns
import scipy.stats as ss
import copy
import time
import networkx as nx
import pandas as pd
import csv
import cPickle as pickle
from collections import Counter
import pathos.multiprocessing as mp

import ising

sns.set_style('white')

ImportError: No module named ising

In [ ]:
reload(ising)

In [ ]:
%load_ext line_profiler

# Some initial runs, tests

In [ ]:
trans = 'async'  # 'single' means that a random single spin is selected to select its new state (according to Glauber)

In [ ]:
T = 1.
system = ising.IsingNetwork(graph=0.3, size=10, T=T, weights=[-1,1], hs=[-1,1])  # weighted, pos and neg
# system = ising.IsingNetwork(graph=0.3, size=10, T=T, weights=1, hs=1)  # unweighted

In [ ]:
print system.states
system.next(trans=trans)
print system.states

In [ ]:
system.graph_numbered.edges(data=True)

In [ ]:
system.graph_numbered.nodes(data=True)

In [ ]:
sns.distplot(map(lambda x: x[1]['h'], system.graph_numbered.nodes(data=True)))
sns.plt.xlabel('External magnetization coefficients $h_i$ per node')
sns.plt.show()

In [ ]:
sns.distplot(map(lambda x: x[2]['weight'], system.graph_numbered.edges(data=True)))
sns.plt.xlabel('Edge weight $J_ij$ per edge')
sns.plt.show()

In [ ]:
nsteps = 2000

system.T = 0.1*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size, +system.size])
sns.plt.show()

system.T = 0.5*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size, +system.size])
sns.plt.show()

system.T = T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size, +system.size])
sns.plt.show()

system.T = 2*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size, +system.size])
sns.plt.show()

system.T = T

## Test

In [ ]:
time_before = time.time()

In [ ]:
_statespace = [1, -1]

In [ ]:
def entropy(ar, base=2):
    cts = np.array(Counter(ar).values())
    cts = cts / float(len(cts))

    return ss.entropy(cts, base=base)

In [ ]:
num_steps = 100
num_cur_states = 100
num_repeats = 100

ising_net = ising.IsingNetwork(size=10, T=1, weights=[-1,1], hs=[-1,1])

In [ ]:
cur_states = []
ising_net2 = copy.deepcopy(ising_net)
for cix in range(num_cur_states):
    ising_net2.reset_states()  # first randomize states again
    cur_states.append(ising_net2.equilibrate(trans=trans))
del ising_net2

In [ ]:
sns.plt.hist(map(sum, cur_states), 2*ising_net.size + 1)
sns.plt.show()

In [ ]:
sns.plt.hist(map(sum, ising_net.go(100000, trans=trans)), 2*ising_net.size + 1)
sns.plt.show()

The two histograms above should look roughly similar

In [ ]:
pre_sum_cur = np.sum(cur_states)  # to be verified remains the same at the end

In [ ]:
np.shape(ising_net.go(num_steps, trans=trans))

In [ ]:
# shape: (num_cur_states, num_repeats, num_steps+1, len(ising_net.states))
states_sequences = [[[cs] + ising_net.go(num_steps, cur_states=cs, trans=trans)
                            for _ in range(num_repeats)]
                    for cs in cur_states]

In [ ]:
np.shape(states_sequences)

In [ ]:
# shape: (len(ising_net.states), num_steps+1, num_cur_states, num_repeats)
states_sequences_t = np.transpose(states_sequences, axes=[3, 2, 0, 1])

In [ ]:
np.shape(states_sequences_t)

In [ ]:
# shape: (len(ising_net.states), num_steps+1, num_cur_states * num_repeats)
states_sequences_t_eq = np.reshape(states_sequences_t, states_sequences_t.shape[:2] + (np.product(states_sequences_t.shape[2:]),))

In [ ]:
# shape: (size, num_steps)
H_eq = [[entropy(states_sequences_t_eq[nix][six]) for six in xrange(num_steps+1)] for nix in xrange(ising_net.size)]

In [ ]:
for nix in xrange(ising_net.size):
    sns.plt.plot(H_eq[nix])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Entropy H[x(t)]')
sns.plt.show()

In [ ]:
# shape: (num_cur_states, size, num_steps)
# this is H[x_i(t) | X[0]]
H_cond_X0 = [[[entropy(states_sequences_t[nix][six][cix])
              for six in xrange(num_steps+1)]
             for nix in xrange(ising_net.size)]
            for cix in xrange(num_cur_states)]

In [ ]:
# shape: (size, _statespace, # cur states where x0=x)
cur_state_ids_where_node_is = [[[cix for cix in xrange(num_cur_states) if cur_states[cix][nix] == x0]
                                for x0 in _statespace]
                               for nix in xrange(ising_net.size)]

In [ ]:
# shape: (_statespace, size)
P_node_x0 = [[float(len(cur_state_ids_where_node_is[nix][xix])) / len(cur_states)
              for nix in xrange(ising_net.size)]
             for xix in xrange(len(_statespace))]

In [ ]:
# shape: (_statespace, size, num_steps)
# note: the whole "np.reshape(...)" is essentially `states_sequences_t_eq` but then conditioned on node `nix` having
# state x0 (xix) in the 'current' state
H_cond_x0 = [[[entropy(np.reshape(np.take(states_sequences_t, cur_state_ids_where_node_is[nix][xix], axis=2),
                                  states_sequences_t.shape[:2] + (len(cur_state_ids_where_node_is[nix][xix]) * num_repeats,))[nix][six])
              if len(cur_state_ids_where_node_is[nix][xix]) > 0 else 0
               for six in xrange(num_steps+1)]
              for nix in xrange(ising_net.size)]
             for xix in xrange(len(_statespace))]

In [ ]:
# shape: (size, num_steps)
I_xt_x0 = np.subtract(H_eq, np.transpose(np.sum([P_node_x0[xix] * np.transpose(H_cond_x0[xix])
                                    for xix in xrange(len(_statespace))], axis=0)))

In [ ]:
print np.shape(H_eq)
print np.shape(H_cond_x0)

In [ ]:
np.shape(P_node_x0[0] * np.transpose(H_cond_x0[0]))

In [ ]:
np.shape(I_xt_x0)

In [ ]:
np.shape(H_cond_X0)

In [ ]:
for nix in xrange(ising_net.size):
    sns.plt.plot(I_xt_x0[nix])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[x(0), x(t)]')
sns.plt.show()

for nix in xrange(ising_net.size):
    sns.plt.plot(I_xt_x0[nix][:min(10, len(I_xt_x0[nix]))])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[x(0), x(t)]')
sns.plt.title('Zoom in')
sns.plt.show()

In [ ]:
# NOTE: assuming here that each system state is equally likely!
# TODO: weigh by likelihoods?
tot_L = np.sum([ising.likelihood(ising_net.graph_numbered, cur_states[cix], ising_net.T) for cix in xrange(len(cur_states))])
# shape: (size, num_steps)
I_xt_X0 = np.subtract(H_eq, np.transpose(np.sum([ising.likelihood(ising_net.graph_numbered, cur_states[cix], ising_net.T) / tot_L * np.transpose(H_cond_X0[cix])
                                    for cix in xrange(len(cur_states))], axis=0)))

In [ ]:
Ls = [ising.likelihood(ising_net.graph_numbered, cur_states[cix], ising_net.T) for cix in xrange(len(cur_states))]

In [ ]:
for nix in xrange(ising_net.size):
    sns.plt.plot(I_xt_X0[nix][:])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[X(0), x(t)]')
sns.plt.show()

for nix in xrange(ising_net.size):
    sns.plt.plot(I_xt_X0[nix][:][:min(20,len(I_xt_X0[nix][:]))])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[X(0), x(t)]')
sns.plt.title('Zoomed in')
sns.plt.show()

In [ ]:
ising.likelihood(ising_net.graph_numbered, cur_states[0], ising_net.T)

In [ ]:
# check equality to histogram above, looking for bug
sns.plt.hist(map(sum, cur_states), 2*ising_net.size + 1)
sns.plt.show()

In [ ]:
assert pre_sum_cur == np.sum(cur_states), 'cur_states changed! should not be!'  # should be same as above

In [ ]:
print time.time() - time_before, 'seconds'

In [ ]:
idt_resp = ising.ce.rel_idts_raw(I_xt_X0)

In [ ]:
sns.distplot(idt_resp.decay_times)
sns.plt.xlabel('IDT of a node')
sns.plt.show()

In [ ]:
time_before = time.time()
idt_resp2 = ising.rel_idt_per_node(system, 'single')
print time.time() - time_before

In [ ]:
sns.distplot(idt_resp2.decay_times)
sns.plt.xlabel('IDT of a node')
sns.plt.show()

## Test the nudging

In [ ]:
reload(ising)

In [ ]:
T = 1.

graph = nx.empty_graph(create_using=nx.DiGraph())
graph.add_node(0, {'h': 0.0})
graph.add_node(1, {'h': 0.0})
graph.add_edge(0, 1, {'weight': 1.0})  # small cycle, keeps the system relatively stable in a magnetization
graph.add_edge(1, 0, {'weight': 1.0})

system = ising.IsingNetwork(graph=graph, T=T, directed=True)  # weighted, pos and neg
# system = ising.IsingNetwork(graph=0.3, size=10, T=T, weights=1, hs=1)  # unweighted

In [ ]:
system.graph_numbered.edges(data=True)

In [ ]:
system.graph_numbered.nodes(data=True)

In [ ]:
%time ss = system.go(10000, trans=trans)

In [ ]:
system.states

In [ ]:
system.reset_states(states=[1, 1])
print system.states

In [ ]:
print 'node 0:'
print ising.energy_node(system.graph_numbered, 0, system.states, T=1.0, state=-1), ising.energy_node(system.graph_numbered, 0, system.states, T=1.0, state=1)
print ising.prob_state(system.graph_numbered, 0, system.states, T=1.0, state=-1), ising.prob_state(system.graph_numbered, 0, system.states, T=1.0, state=1)
print 'node 1:'
print ising.energy_node(system.graph_numbered, 1, system.states, T=1.0, state=-1), ising.energy_node(system.graph_numbered, 1, system.states, T=1.0, state=1)
print ising.prob_state(system.graph_numbered, 1, system.states, T=1.0, state=-1), ising.prob_state(system.graph_numbered, 1, system.states, T=1.0, state=1)

In [ ]:
nsteps = 3000

system.T = 0.1*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = 0.5*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = 2*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(sum, system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = T

In [ ]:
system.add_nudge_to(1, (0., 2.))

In [ ]:
system.graph_numbered.edges(data=True)

In [ ]:
system.graph_numbered.nodes(data=True)

In [ ]:
# system.graph_numbered.remove_edge(-1, 1)

In [ ]:
system.states

In [ ]:
%time ss = system.go(10000, trans=trans)

In [ ]:
system.go(3, trans=trans)

In [ ]:
system.reset_states(states=[1, 1])
print system.states

In [ ]:
print 'node 0:'
print ising.energy_node(system.graph_numbered, 0, system.states, T=1.0, state=-1), ising.energy_node(system.graph_numbered, 0, system.states, T=1.0, state=1)
print ising.prob_state(system.graph_numbered, 0, system.states, T=1.0, state=-1), ising.prob_state(system.graph_numbered, 0, system.states, T=1.0, state=1)
print 'node 1:'
print ising.energy_node(system.graph_numbered, 1, system.states, T=1.0, state=-1), ising.energy_node(system.graph_numbered, 1, system.states, T=1.0, state=1)
print ising.prob_state(system.graph_numbered, 1, system.states, T=1.0, state=-1), ising.prob_state(system.graph_numbered, 1, system.states, T=1.0, state=1)

In [ ]:
nsteps = 3000

system.T = 0.1*T

# TODO: nudger nodes should not be taken into account when computing magnetization etc.

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(lambda d: sum(d.itervalues()), system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = 0.5*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(lambda d: sum(d.itervalues()), system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(lambda d: sum(d.itervalues()), system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = 2*T

sns.plt.figure(figsize=(21, 8))
sns.plt.plot(map(lambda d: sum(d.itervalues()), system.go(nsteps, trans=trans)))
sns.plt.ylim([-system.size - 0.1, +system.size + 0.1])
sns.plt.show()

system.T = T

## Test state distributions

In [ ]:
reload(ising)

In [ ]:
T = 1.

graph = nx.empty_graph(create_using=nx.DiGraph())
graph.add_node(0, {'h': 0.0})
graph.add_node(1, {'h': 0.0})
graph.add_edge(0, 1, {'weight': 1.0})  # small cycle, keeps the system relatively stable in a magnetization
graph.add_edge(1, 0, {'weight': 1.0})

system = ising.IsingNetwork(graph=graph, T=T, directed=True)  # weighted, pos and neg
# system = ising.IsingNetwork(graph=0.3, size=10, T=T, weights=1, hs=1)  # unweighted

In [ ]:
system.graph_numbered.edges(data=True)

In [ ]:
system.graph_numbered.nodes(data=True)

In [ ]:
system.reset_states([1, 1])
print system.state_distributions(30000, trans='async')
system.reset_states([1, 1])
print system.state_distributions(30000, trans='async')  # repeat a second time to eyeball consistency

In [ ]:
system.add_nudge_to(0, (0, 1.0))

In [ ]:
# state distributions after nudge
system.reset_states([1, 1])
print system.state_distributions(30000, trans='async')
system.reset_states([1, 1])
print system.state_distributions(30000, trans='async')  # repeat a second time to eyeball consistency

In [ ]:
system.remove_all_nudges()

In [ ]:
%time system.impact_of_nudge(0, (0.1,  0.0))

In [ ]:
# try multiple same impact calculations to get an idea of the variance
impacts = [system.impact_of_nudge(0, (0.1,  0.0), steps_per_node=10000) for _ in xrange(10)]

In [ ]:
sns.distplot(impacts, rug=True)
sns.plt.show()

## Test nudge impact in a network

In [ ]:
graph = ising.generate_directed_powerlaw_network(10, 2.0)

print graph.nodes()

# add also a disconnected node so that we can test that its nudge impact is indeed zero
graph.add_node(graph.number_of_nodes())

print graph.nodes()

In [ ]:
graph.number_of_nodes()

In [ ]:
# shorthand for all the weights in the network
weights = np.array([a['weight'] if a.has_key('weight') else 1.0 
                    for (_,_,a) in graph.edges(data=True)])

# sns.distplot(weights)
# sns.plt.xlabel('weights')
# sns.plt.show()

# position nodes in the 2D plane using some spring force computation using the weights
pos = nx.spring_layout(graph, iterations=20000);

edge_label_scale = 100.
edge_labels = {(e[0], e[1]): str(round((e[2]['weight'] if e[2].has_key('weight') else 1.0) * edge_label_scale, 1))
               for e in graph.edges(data=True)};

fig = sns.plt.figure(1, figsize=(12,12));
fig.clear();
sns.plt.axis('off');
nx.draw_networkx_nodes(graph, pos, 
                       node_size=300., 
                       node_color=[np.array([0., 0., 0.]) for n1 in graph.nodes_iter()]);
nx.draw_networkx_labels(graph, {k: v + [0, +0.03] for (k,v) in pos.iteritems()}, font_weight='bold', 
                       font_size=16);
# nx.draw_networkx_edge_labels(graph, pos, edge_labels=edge_labels);
edge_color_baseline = 0.5
edge_width_exp = 2.0
nx.draw_networkx_edges(graph, pos, width=np.power(weights, edge_width_exp) / np.mean(np.power(weights, edge_width_exp)))
#                        edge_color=map(str, edge_color_baseline*np.subtract(weights, min(weights))/max(weights)));

# if save_figs:
# #     fn = op.join(save_dir, 'skew_diffs_forwardsearch_%srepeats_%s_%s_alpha%s_kz%s_mz%s_uniz%s.pdf' % (num_repeats, X, Y, alpha_multi_fw, kz, mz, int(sample_Z_uniformly)))
#     fn = op.join(save_dir, filename_tagged('causal_graph_exp%s' % edge_width_exp, pair=False))
#     sns.plt.savefig(fn)
#     print 'note: saved to', fn
# nx.write_graph6(causal_graph, op.join(save_dir, filename_tagged('causal_graph_exp%s' % edge_width_exp, ext='graph6', pair=False)))
# print 'note: graph saved to', op.join(save_dir, filename_tagged('causal_graph_exp%s' % edge_width_exp, ext='graph6', pair=False))
sns.plt.show()

In [ ]:
system = ising.IsingNetwork(graph, graph.number_of_nodes(), T=2.0, states=np.ones(graph.number_of_nodes()))

In [ ]:
assert len(system.graph_numbered.neighbors(graph.number_of_nodes()-1)) == 0

In [ ]:
Ts, Hs = system.entropy_next_vs_temp()

sns.plt.plot(Ts, Hs)
sns.plt.xlabel('T')
sns.plt.ylabel('<H(s)>')
sns.plt.show()

In [ ]:
T = system.temp_matching_entropy(0.5)
T

In [ ]:
system.T = T

In [ ]:
np.sum(system.equilibrate()) / system.size

In [ ]:
system.graph_numbered.predecessors(system.size - 1), system.graph_numbered.successors(system.size - 1)

In [ ]:
%time system.impact_of_nudge(system.size - 1, [0, 1.0], steps_per_node=5000, include_self=False)

In [ ]:
def hellinger_null_distribution(n, p, ntrials=20):
    hells = []
    for i in xrange(ntrials):
        succp = np.random.binomial(n, p) / float(n)
        failp = 1.0 - succp
        succp2 = np.random.binomial(n, p) / float(n)
        failp2 = 1.0 - succp2
        
        hells.append(ising.hellinger_distance([succp, failp], [succp2, failp2]))
    return hells

In [ ]:
p = np.mean(system.state_distributions(2), axis=0)[0]
ns = [500, 1000, 2000, 3000, 5000, 10000]
null_hell_per_n = [hellinger_null_distribution(n, p) for n in ns]

In [ ]:
sns.plt.errorbar(ns, np.mean(null_hell_per_n, axis=1), yerr=np.std(null_hell_per_n, axis=1) / np.sqrt(1))
sns.plt.xlabel('Number of samples to estimate state prob.')
sns.plt.ylabel('Expected Hellinger distance per node (=nudge impact / size)')
sns.plt.title('Null distribution when sampling two identical binomial distributions')
sns.plt.show()

In [ ]:
p = 0.5
ns = [500, 1000, 2000, 3000, 5000, 10000]
null_hell_per_n = [hellinger_null_distribution(n, p) for n in ns]

In [ ]:
sns.plt.errorbar(ns, np.mean(null_hell_per_n, axis=1), yerr=np.std(null_hell_per_n, axis=1) / np.sqrt(1))
sns.plt.xlabel('Number of samples to estimate state prob.')
sns.plt.ylabel('Expected Hellinger distance per node (=nudge impact / size)')
sns.plt.title('Null distribution when sampling two identical binomial distributions (p=0.5)')
sns.plt.show()

### Check performance profile of computing these impacts, they take a bit long

In [ ]:
#%lprun -f ising.total_network_interaction_energy_node system.impact_of_nudge(0, [0, 1.0], steps_per_node=1000)

### Compute impact for each node with the same nudge

In [ ]:
def worker_impact(nix):
    return system.impact_of_nudge(nix, [0, 1.0], steps_per_node=20000, include_self=False)

time_before = time.time()

pool = mp.Pool(6)
impacts = pool.map(worker_impact, graph.nodes())
pool.close()
pool.terminate()

print (-time_before + time.time()) / 60., 'minutes'

In [ ]:
sns.plt.bar(range(system.size), impacts)
sns.plt.show()

### Compare with network topology features

In [ ]:
weighted_degrees = [system.graph_numbered.degree(nix, weight='weight') for nix in range(system.size)]
weighted_out_degrees = [system.graph_numbered.out_degree(nix, weight='weight') for nix in range(system.size)]
weighted_in_degrees = [system.graph_numbered.in_degree(nix, weight='weight') for nix in range(system.size)]

In [ ]:
bcdist = nx.centrality.betweenness_centrality(system.graph_numbered, weight='weight')
bcs = [bcdist[nix] for nix in range(system.size)]

In [ ]:
measures_list = [weighted_degrees, weighted_out_degrees, weighted_in_degrees, bcs, np.subtract(weighted_out_degrees, weighted_in_degrees), np.add(weighted_out_degrees, weighted_in_degrees)]
xlabels_list = ['weighted_degrees', 'weighted_out_degrees', 'weighted_in_degrees', 'bcs', 'out - in', 'out + in']

sns.plt.figure(figsize=(15, 15))

for mix, measures in enumerate(measures_list):
    sns.plt.subplot(3, 2, len(measures_list) - mix)
    sns.distplot(measures)
    sns.plt.xlabel(xlabels_list[mix])

sns.plt.show()

In [ ]:
# measures_list = [weighted_degrees, weighted_out_degrees, weighted_in_degrees, bcs, np.subtract(weighted_out_degrees, weighted_in_degrees)]
# xlabels_list = ['weighted_degrees', 'weighted_out_degrees', 'weighted_in_degrees', 'bcs', 'out - in']

sns.plt.figure(figsize=(15, 15))

for mix, measures in enumerate(measures_list):
    sns.plt.subplot(3, 2, len(measures_list) - mix)
    sns.plt.plot(measures, impacts, 'o', markersize=15, alpha=0.5)
    sns.plt.xlabel(xlabels_list[mix])
    sns.plt.ylabel('Impact of nudge')
    if not xlabels_list[mix] in ('out - in',):
        sns.plt.xscale('log')

sns.plt.show()

### Compare with IDT

In [ ]:
time_before = time.time()
resp = ising.idt_per_node(system, 'async', num_cur_states=300, num_repeats=300, num_steps=50, 
                          assume_symmetry=False, include_double_exp=True, nprocs=5)
print time.time() - time_before, 'seconds'

In [ ]:
startix = 0  # keep at 0

for nix in xrange(system.size):
    sns.plt.plot(resp.mi_over_time[nix][startix:])
sns.plt.xlabel('Time step $t$')
sns.plt.ylabel('Mutual information I[X(0), x(t)]')
# sns.plt.yscale('log')
sns.plt.savefig('./mi_over_time_per_node_full_T%s_%s.png' % (system.T, 'async'))
sns.plt.show()

for nix in xrange(system.size):
    sns.plt.plot(resp.mi_over_time[nix][startix:])
sns.plt.xlabel('Time step $t$')
sns.plt.ylabel('Mutual information I[X(0), x(t)]')
sns.plt.yscale('log')
sns.plt.title('Log-linear')
sns.plt.savefig('./mi_over_time_per_node_full_T%s_%s.png' % (system.T, 'async'))
sns.plt.show()

for nix in xrange(system.size):
    sns.plt.plot(resp.mi_over_time[nix][startix:][:min(20,len(resp.mi_over_time[nix][startix:]))])
sns.plt.xlabel('Time step $t$')
sns.plt.ylabel('Mutual information I[X(0), x(t)]')
sns.plt.title('Zoomed in')
sns.plt.savefig('./mi_over_time_per_node_firstpart_T%s_%s.png' % (system.T, 'async'))
sns.plt.show()

In [ ]:
idts_list = [resp.decay_times, resp.decay_times_abs, resp.asymptotic_values]
xlabels_idts_list = ['Rel. IDT', 'Abs. IDT', 'Asympt. MI']

sns.plt.figure(figsize=(15, 15))

for mix, measures in enumerate(idts_list):
    sns.plt.subplot(3, 2, len(idts_list) - mix)
    sns.plt.plot(measures, impacts, 'o', markersize=15, alpha=0.5)
    sns.plt.xlabel(xlabels_idts_list[mix])
    sns.plt.ylabel('Impact of nudge')
    if xlabels_idts_list[mix] in ('Rel. IDT',):
        sns.plt.xscale('log')

sns.plt.show()

#### Intermezzo: fit a double exponential decay

In [ ]:
def residuals_double_decay(s, xdata, ydata):
    '''
    Residuals of the curve a*exp(-b*x) + c*exp(-d*x) + plateau
    @ s: (a, b, c, d, plateau)
    @return: residuals of fitting to xdata, ydata
    '''
    a, b, c, d, plateau = s
    return np.linalg.norm(a*np.exp(-b*xdata) + c*np.exp(-d*xdata) + plateau - ydata)

def values_double_decay(s, xdata):
    a, b, c, d, plateau = s
    return a*np.exp(-b*xdata) + c*np.exp(-d*xdata) + plateau

In [ ]:
import scipy.optimize as sopt

In [ ]:
def fit_double_exp_decay(mi_over_time):
    '''
    Residuals of the curve a*exp(-b*x) + c*exp(-d*x) + plateau
    @ s: (a, b, c, d, plateau)
    @return: (a, b, c, d, plateau), making sure that b >= d (so the first exponential is the fast decay).
    To calculate the two corresponding relative IDTs (fraction 1/e) you simply do 1/b and 1/d. For the
    absolute IDTs to MI=eps do -1/b * np.log(eps/a) and -1/d * np.log(eps/c)
    '''
    optres = sopt.minimize(residuals_double_decay, 
                       x0=[0.1, 3., 0.9, 1., 0.],
                       args=(np.arange(len(mi_over_time)), np.array(mi_over_time)), 
                       bounds=[(0., max(mi_over_time)), (0., 100.),
                              (0., max(mi_over_time)), (0., 100.),
                              (0., max(mi_over_time))])
    
    if optres.success:
        a, b, c, d, plateau = optres.x  # readability
        if b < d:
            return c, d, a, b, plateau
        else:
            return a, b, c, d, plateau
    else:
        raise UserWarning('could not find solution')
        

def convert_double_exp_s_to_idts(s, epsilon=0.001):
    '''
    Convert the result of fit_double_exp_decay() to IDT (or equivalently IDL) values, which may be 
    more meaningful to your users as the units are number of time steps.
    @s s: result from fit_double_exp_decay()
    @ epsilon: a very small absolute and positive amount of mutual information
    @return: (rel IDT 1, rel IDT 2, abs IDT 1, abs IDT 2). 
    Due to fit_double_exp_decay()'s ordering, rel IDT 2 >= rel IDT 1.
    '''
    a, b, c, d, plateau = s
    return 1/b, 1/d, -1/b * np.log(epsilon / a), -1/d * np.log(epsilon / c)

In [ ]:
mis = resp.mi_over_time[6]
sol = fit_double_exp_decay(mis)
print sol
sns.plt.plot(mis, 'o')
sns.plt.plot(values_double_decay(sol, np.arange(len(mis))))
sns.plt.show()

In [ ]:
# Rel. IDTs (1/e):
1/sol[1], 1/sol[3]

In [ ]:
# Abs. IDTs (1/e):
-1/sol[1] * (np.log(0.001 / sol[0])), -1/sol[3] * (np.log(0.001 / sol[2]))

In [ ]:
if not hasattr(resp, 'double_exp_idts'):
    print 'note: will also compute the double exponential IDTs'
    double_exp_s = []
    double_exp_idts = []
    # as reminder to the caller in what order the IDTs are given in double_exp_idts
    double_idt_labels = ('relIDTfast', 'relIDTslow', 'absIDTfast', 'absIDTslow')

    for mis in resp.mi_over_time:
        s = fit_double_exp_decay(mis)
        # expanded for readability:
        relIDT1, relIDT2, absIDT1, absIDT2 = convert_double_exp_s_to_idts(s)
        double_exp_s.append(s)
        double_exp_idts.append((relIDT1, relIDT2, absIDT1, absIDT2))

    resp.double_exp_s = double_exp_s
    resp.double_exp_idts = double_exp_idts
    resp.double_idt_labels = double_idt_labels

### Compare to double expontial decay IDTs

In [ ]:
np.shape(list(np.transpose(resp.double_exp_s)[np.array((0,2))]))

In [ ]:
idts2_list = list(np.transpose(resp.double_exp_idts)) \
            + list([np.transpose(resp.double_exp_s)[-1]]) \
            + list(np.transpose(resp.double_exp_s)[np.array((0,2))]) \
            + list([1.0/(np.transpose(resp.double_exp_s)[2]/np.transpose(resp.double_exp_s)[0])])
xlabels2_idts_list = ['relIDT1', 'relIDT2', 'absIDT1', 'absIDT2', 'plateau', 'weight fast', 'weight slow', 'ratio fast/slow weights']

sns.plt.figure(figsize=(15, 15))

for mix, measures in enumerate(idts2_list):
    sns.plt.subplot(4, 2, len(idts2_list) - mix)
    sns.plt.plot(measures, impacts, 'o', markersize=15, alpha=0.5)
    sns.plt.xlabel(xlabels2_idts_list[mix])
    sns.plt.ylabel('Impact of nudge')
    if xlabels2_idts_list[mix] in ('absIDT2', 'relIDT2', 'ratio fast/slow weights'):
        sns.plt.xscale('log')

sns.plt.show()

#### Try to find a predictive combination of information measures

In [ ]:
idts2_list = [(np.transpose(resp.double_exp_s)[0]/np.transpose(resp.double_exp_s)[2] * (np.transpose(resp.double_exp_s)[1]+np.transpose(resp.double_exp_s)[3]))]
xlabels2_idts_list = ['slow/fast+idt1+idt2']

sns.plt.figure(figsize=(15, 15))

for mix, measures in enumerate(idts2_list):
    sns.plt.subplot(4, 2, len(idts2_list) - mix)
    sns.plt.plot(measures, impacts, 'o', markersize=15, alpha=0.5)
    sns.plt.xlabel(xlabels2_idts_list[mix])
    sns.plt.ylabel('Impact of nudge')
    if xlabels2_idts_list[mix] in ('slow/fast+idt1+idt2',):
        sns.plt.xscale('log')

sns.plt.show()

#### Try to derive a useful measure

$$ a e^{-bx} + c e^{-dx} + p = p + f \cdot (a + c) $$
$$ a e^{-bx} + c e^{-dx} = f \cdot a + f \cdot c $$
$$ a (e^{-bx} - f) + c (e^{-dx} - f) = 0 $$
$$ a (e^{-bx} - f) = - c (e^{-dx} - f) $$
$$ -\frac{a}{c} (e^{-bx} - f) = e^{-dx} - f $$
$$ -\frac{a}{c} (e^{-bx} - f) + f \frac{a/c}{a/c} = e^{-dx} $$
$$ -\frac{a}{c} (e^{-bx} - f - f \frac{1}{a/c}) = e^{-dx} $$
$$ -\frac{a}{c} (e^{-bx} - f \left(\frac{1}{a/c} + 1\right)) = e^{-dx} $$
$$ \log{-\frac{a}{c} (e^{-bx} - f \left(\frac{1}{a/c} + 1\right))} = -dx $$
$$ -\frac{1}{d} \log{-\frac{a}{c} (e^{-bx} - f \left(\frac{1}{a/c} + 1\right))} = x $$
$$ -\frac{1}{d} \log{\left[-\frac{a}{c} e^{-bx} + \frac{a}{c} f \left(\frac{1}{a/c} + 1\right)\right]} = x $$
$$ \log{\left[-\frac{a}{c} e^{-bx} + f \left(\frac{a}{c} + 1\right)\right]} = d x $$

From this point it is difficult to proceed exactly. Let us do the Taylor expansion trick.

$$ \log{\left[\frac{-\frac{a}{c} e^{-bx}}{f \left(\frac{a}{c} + 1\right)} + 1\right] + \log{f \left(\frac{a}{c} + 1\right)}} = d x $$
$$ \log{\left[\frac{-a/c}{a/c+1} \frac{e^{-bx}}{f} + 1\right] + \log{f} + \log{\left(\frac{a}{c} + 1\right)}} = d x $$

Suppose $f=e^{-1}$, then:

$$ \log{\left[\frac{-a/c}{a/c+1} e^{-bx-1} + 1\right]} - 1 + \log{\left(\frac{a}{c} + 1\right)} = d x $$

Suppose that $\left| \frac{-a/c}{a/c+1} e^{-bx-1} \right| << 1$, which is not that crazy and true at least as $x >> 1$:

$$ \frac{-a/c}{a/c+1} e^{-bx-1} - 1 + \log{\left(\frac{a}{c} + 1\right)} = d x $$

Suppose also that $a/c << 1$:

$$ \frac{-a/c}{a/c+1} e^{-bx-1} - 1 + \frac{a}{c} = d x $$

### Why does IDT take so long?

In [ ]:
reload(ising)

In [ ]:
# %lprun -f ising.idt_per_node ising.idt_per_node(system, 'async', num_cur_states=50, num_repeats=50, num_steps=10, assume_symmetry=False)

### Contrived example for validation

In [ ]:
graph2 =nx.empty_graphpty_graph(using=nx.DiGraph())
for oix in range(1, 5):
    graph2.add_edge(0, oix, {'weight': 1.0})
for oix in range(5, 7):
    graph2.add_edge(5, oix, {'weight': 1.0})

In [ ]:
system2 = ising.IsingNetwork(graph2, graph2.number_of_nodes(), T=2.0, states='random')

# Read weighted Ising network from file

In [ ]:
matrix_weights = pd.read_csv('./Graph_min1_1.csv', index_col=0)

In [ ]:
matrix_weights

In [ ]:
matrix_weights.columns

In [ ]:
matrix_weights['depr']['sad']

In [ ]:
vector_hs = pd.read_csv('./External_min1_1.csv', index_col=0, header=0)
vector_hs = vector_hs['externalField']
vector_hs_list = [vector_hs[col] for col in matrix_weights.columns]

In [ ]:
vector_hs['depr']

In [ ]:
graph = nx.from_numpy_matrix(np.array(matrix_weights), create_using=nx.DiGraph())

In [ ]:
graph.node[5]

In [ ]:
for colix, col in enumerate(matrix_weights.columns):
    graph.node[colix]['h'] = vector_hs_list[colix]

## Calculate IDT on this network

In [ ]:
import pathos.multiprocessing as mp

In [ ]:
T=1.25

In [ ]:
psycho_net = ising.IsingNetwork(T=T)

unidirectional = False

In [ ]:
psycho_net.read_graph_from_csv('./Graph_min1_1.csv', './External_min1_1.csv')

In [ ]:
realization = ''
if unidirectional:
    for nxi, ny in psycho_net.graph_numbered.edges():
    #     if nx < ny:
        # still a bi-directional edge?
        if psycho_net.graph_numbered.has_edge(nxi, ny) and psycho_net.graph_numbered.has_edge(ny, nxi):
            # remove one of the edges
            if np.random.choice([0, 1]) == 1:
                psycho_net.graph_numbered.remove_edge(nxi, ny)
                realization = realization + '1'
            else:
                psycho_net.graph_numbered.remove_edge(ny, nxi)
                realization = realization + '0'
else:
    realization = '1' * psycho_net.graph_numbered.number_of_edges()
    
print realization

In [ ]:
psycho_net.T = T

mags = map(sum, psycho_net.go(2000, trans=trans))

sns.plt.plot(mags)
sns.plt.ylim([-psycho_net.size, +psycho_net.size])
sns.plt.show()

sns.distplot(mags, kde=False)
sns.plt.xlim([-psycho_net.size, +psycho_net.size])
sns.plt.show()

print np.median(mags)
print mags.count(0)

In [ ]:
# psycho_net.reset_states()

In [ ]:
Ts = np.linspace(0.01, 20.0, 10)
medians = []
stds = []
zeros = []

for t in Ts:
    psycho_net.T = t
#     psycho_net.reset_states()
    
    mags = map(sum, psycho_net.go(10000, trans=trans))
    medians.append(np.median(mags))
    stds.append(np.std(mags))
    zeros.append(mags.count(0))
    
    print 'note: finished T=' + str(t)

psycho_net.T = T

In [ ]:
sns.plt.plot(Ts, medians, '-x')
sns.plt.plot(Ts, stds, '-o')
sns.plt.xlabel('Temperature')
sns.plt.show()

In [ ]:
sns.plt.plot(Ts, medians, '-x')
sns.plt.plot(Ts, stds, '-o')
sns.plt.xlabel('Temperature')
sns.plt.show()

sns.plt.plot(Ts, zeros, '-o')
sns.plt.xlabel('Temperature')
sns.plt.ylabel('Number of zero magnetization')
sns.plt.show()

In [ ]:
assume_symmetry = False  # should set to False if there are external magnetic fields, at least (which is symmetry breaking)

In [ ]:
try_to_reuse = True

time_before = time.time()
mi_per_node = None
psycho_idt_resp = None
if try_to_reuse:
    try:
#         print 'debug: will try file:', './psycho_idt_resp_T%s_uni%s.pickle' % (psycho_net.T, int(unidirectional))
        with open('./psycho_idt_resp_T%s_uni%s.pickle' % (psycho_net.T, int(unidirectional)), 'rb') as fin:
            psycho_idt_resp = pickle.load(fin)

        with open('./mi_per_node_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)), 'rb') as fin:
            mi_per_node = pickle.dump(fin)
    except:
        mi_per_node = None
        psycho_idt_resp = None
    else:
        print 'note: reused previously computed result from file', './psycho_idt_resp_T%s_uni%s.pickle' % (psycho_net.T, int(unidirectional))
if mi_per_node is None or psycho_idt_resp is None:
    psycho_idt_resp, mi_per_node = ising.rel_idt_per_node(psycho_net, trans, num_cur_states=300, num_repeats=300, 
                                                          num_steps=70, return_also_mi=True, assume_symmetry=assume_symmetry)
print time.time() - time_before

In [ ]:
for nix in xrange(psycho_net.size):
    sns.plt.plot(mi_per_node[nix][:])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[X(0), x(t)]')
sns.plt.savefig('./mi_over_time_per_node_full_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)))
sns.plt.show()

for nix in xrange(psycho_net.size):
    sns.plt.plot(mi_per_node[nix][:][:min(20,len(mi_per_node[nix][:]))])
sns.plt.xlabel('Time step $t$')
sns.plt.xlabel('Mutual information I[X(0), x(t)]')
sns.plt.title('Zoomed in')
sns.plt.savefig('./mi_over_time_per_node_firstpart_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)))
sns.plt.show()

In [ ]:
idt_per_node = zip(matrix_weights.columns, psycho_idt_resp.decay_times)

with open('./idt_per_node_T%s_uni%s.csv' % (psycho_net.T, int(unidirectional)), 'wb') as fout:
    csvw = csv.writer(fout)
    csvw.writerows(idt_per_node)


In [ ]:
weighted_degrees = [psycho_net.graph_numbered.degree(nix, weight='weight') for nix in range(psycho_net.size)]
weighted_out_degrees = [psycho_net.graph_numbered.out_degree(nix, weight='weight') for nix in range(psycho_net.size)]
weighted_in_degrees = [psycho_net.graph_numbered.in_degree(nix, weight='weight') for nix in range(psycho_net.size)]

In [ ]:
bcdist = nx.centrality.betweenness_centrality(psycho_net.graph_numbered, weight='weight')
bcs = [bcdist[nix] for nix in range(psycho_net.size)]

In [ ]:
psycho_net.graph_numbered.out_degree(0, weight='weight')

In [ ]:
sns.plt.plot(weighted_degrees, psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Weighted degree')
sns.plt.ylabel('IDT')
sns.plt.savefig('./scatter_idt_vs_weighted_degree_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)))
sns.plt.show()

sns.plt.plot(np.abs(weighted_degrees), psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Abs[Weighted degree]')
sns.plt.ylabel('IDT')
sns.plt.savefig('./scatter_idt_vs_abs_weighted_degree_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)))
sns.plt.show()

sns.plt.plot(weighted_out_degrees, psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Weighted out-degree')
sns.plt.ylabel('IDT')
sns.plt.show()

sns.plt.plot(np.abs(weighted_out_degrees), psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Abs[Weighted out-degree]')
sns.plt.ylabel('IDT')
sns.plt.show()

sns.plt.plot(weighted_in_degrees, psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Weighted in-degree')
sns.plt.ylabel('IDT')
sns.plt.show()

sns.plt.plot(np.abs(weighted_in_degrees), psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Abs[Weighted in-degree]')
sns.plt.ylabel('IDT')
sns.plt.show()

print '----------------'

sns.plt.plot(bcs, psycho_idt_resp.decay_times, 'o')
sns.plt.xlabel('Betweenness centrality (weighted)')
sns.plt.ylabel('IDT')
sns.plt.savefig('./scatter_idt_vs_betweenness_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)))
sns.plt.show()

In [ ]:
with open('./psycho_idt_resp_T%s_uni%s.pickle' % (psycho_net.T, int(unidirectional)), 'wb') as fout:
    pickle.dump(psycho_idt_resp, fout)
    
with open('./mi_per_node_T%s_uni%s.png' % (psycho_net.T, int(unidirectional)), 'wb') as fout:
    pickle.dump(mi_per_node, fout)

# Random networks

Identify the largest driver node (largest IDT) in randomly generated networks and study the topological features of this node.

In [ ]:
nsys = 10
